# Program - 

**Purpose**


**Content**
- read data
- plot XY profiles

**Author:** Yi-Hsuan Chen (yihsuan@umich.edu)

**Date:** 

**Reference program:**



In [2]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import io, os, sys, types

#import yhc_module as yhc

xr.set_options(keep_attrs=True)  # keep attributes after xarray operation

## Read data

### function - get_hindcast_filenames

In [48]:
from datetime import datetime, timedelta

############################
############################
############################
def get_previous_date(input_date, days_prior):
    """
    Get the date prior to the input_date

    Input argument:
        input_date = '2001-01-04'
        days_prior = 3

    Output argument:
        a dictionary variable [date, YYYY, MM, DD, YYYYMMDD]

    Example:
        input_date = '2001-01-04'
        days_prior = 3
        date_prior3 = get_previous_date(input_date, days_prior)
        
        date_prior3['date'] is 2001-01-01
        
    Author: Yi-Hsuan Chen
    January 2024
    """
    # Convert the input date string to a datetime object
    input_date = datetime.strptime(input_date, '%Y-%m-%d')

    # Calculate the timedelta for the specified number of days
    delta = timedelta(days=days_prior)

    # Subtract the timedelta from the input date to get the previous date
    previous_date = input_date - delta

    # Format the result as a string in the same format as the input
    date = previous_date.strftime('%Y-%m-%d')
    year, month, day = date.split('-')

    result = {
        'date':date,
        'YYYY':year,
        'MM':month,
        'DD':day,
        #'date_merge':str(year)+str(month).zfill(2)+str(day).zfill(2)
        'YYYYMMDD':year+month+day
             }
    return result

############################
############################
############################
def get_hindcast_filenames(start_date_str, end_date_str, lead_day,
                           filename_head="xx01-taiesm1.F_2000_TAI.f09_f09.1226_1050.icdate_"):

    """
    Get hindcast simulation file names

    Input:
        start_date_str = '2001-07-01'
        end_date_str = '2001-07-31'
        lead_day=2

    Output:
        filenames, e.g. {filename_head}.20010629.cam.h0.2001-07-01-00000.nc
    """
    
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    # Define the timedelta for the loop iteration
    delta = timedelta(days=1)

    # Initialize the current date with the start date
    current_date = start_date

    # Loop over the date range

    filenames = []
    
    while current_date <= end_date:
        current_date_str = current_date.strftime('%Y-%m-%d')
        date00 = get_previous_date(current_date_str, lead_day)
        
        #print(date00['date_merge'])

        yyyy, mm ,dd = current_date_str.split('-')
        filename1 = filename_head+date00['YYYYMMDD']+".cam.h0."+current_date_str+"-00000.nc"

        #print(filename1)
        filenames.append(filename1)
        
        # Move to the next date
        current_date += delta

    return filenames
    #print(filenames)

#------------
# do test
#-------------
start_date_str = '2001-07-01'
end_date_str = '2001-07-31'
lead_day=2
filenames = get_hindcast_filenames(start_date_str, end_date_str, lead_day)

#filenames

In [49]:
start_date_str = '2001-07-01'
end_date_str = '2001-07-31'
lead_day=2
filenames = get_hindcast_filenames(start_date_str, end_date_str, lead_day)

filenames

['xx01-taiesm1.F_2000_TAI.f09_f09.1226_1050.icdate_20010629.cam.h0.2001-07-01-00000.nc',
 'xx01-taiesm1.F_2000_TAI.f09_f09.1226_1050.icdate_20010630.cam.h0.2001-07-02-00000.nc',
 'xx01-taiesm1.F_2000_TAI.f09_f09.1226_1050.icdate_20010701.cam.h0.2001-07-03-00000.nc',
 'xx01-taiesm1.F_2000_TAI.f09_f09.1226_1050.icdate_20010702.cam.h0.2001-07-04-00000.nc',
 'xx01-taiesm1.F_2000_TAI.f09_f09.1226_1050.icdate_20010703.cam.h0.2001-07-05-00000.nc',
 'xx01-taiesm1.F_2000_TAI.f09_f09.1226_1050.icdate_20010704.cam.h0.2001-07-06-00000.nc',
 'xx01-taiesm1.F_2000_TAI.f09_f09.1226_1050.icdate_20010705.cam.h0.2001-07-07-00000.nc',
 'xx01-taiesm1.F_2000_TAI.f09_f09.1226_1050.icdate_20010706.cam.h0.2001-07-08-00000.nc',
 'xx01-taiesm1.F_2000_TAI.f09_f09.1226_1050.icdate_20010707.cam.h0.2001-07-09-00000.nc',
 'xx01-taiesm1.F_2000_TAI.f09_f09.1226_1050.icdate_20010708.cam.h0.2001-07-10-00000.nc',
 'xx01-taiesm1.F_2000_TAI.f09_f09.1226_1050.icdate_20010709.cam.h0.2001-07-11-00000.nc',
 'xx01-taiesm1.F_2000

## Plot - 

In [23]:
#yhc.lib("fdef")   # check out yhc library